In [ ]:
# LangGraph Agentic Workflow for Supply Chain with Chat History + Firecrawl + MongoDB

from typing_extensions import TypedDict, NotRequired
from langgraph.graph import StateGraph, START, END
from langchain.tools import tool
from pymongo import MongoClient
from datetime import datetime
import requests

# 1. Shared State
class CreateChainState(TypedDict):
    session_id: str
    raw_input: dict
    requirements: dict
    esg_preference: NotRequired[str]
    suppliers: list
    exploration_results: list
    evaluation_feedback: str
    final_report: dict

# 2. MongoDB Setup
MONGO_URI = "mongodb+srv://<username>:<password>@<cluster>.mongodb.net/?retryWrites=true&w=majority"
client = MongoClient(MONGO_URI)
db = client["supply_chain"]
suppliers_collection = db["suppliers"]
conversation_collection = db["chat_history"]
report_collection = db["final_reports"]

# 3. Log chat history

def log_message(role: str, content: str, session_id: str):
    conversation_collection.insert_one({
        "session_id": session_id,
        "role": role,
        "content": content,
        "timestamp": datetime.utcnow()
    })

# 4. Requirement Analysis Agent

def parse_requirements(state: CreateChainState):
    session_id = state["session_id"]
    user_input = state["raw_input"]
    log_message("user", str(user_input), session_id)

    search_terms = " ".join(str(v) for v in user_input.values() if isinstance(v, str))
    history_cursor = conversation_collection.find({"$text": {"$search": search_terms}, "role": "user"}).sort("timestamp", -1).limit(5)
    combined_requirements = dict(user_input)

    for msg in history_cursor:
        try:
            past_req = eval(msg["content"])
            if isinstance(past_req, dict):
                combined_requirements.update(past_req)
        except:
            pass

    return {"requirements": combined_requirements}

# 5. MongoDB Tool
@tool
def query_mongodb(requirements: dict, esg_filter: str = None) -> list:
    query = dict(requirements)
    if esg_filter:
        query["esg_score"] = esg_filter
    return list(suppliers_collection.find(query, {"_id": 0}))

# 6. Firecrawl Tool
@tool
def firecrawl_scrape_tool(requirements: dict) -> list:
    FIRECRAWL_API_KEY = "your_firecrawl_api_key"
    FIRECRAWL_URL = "https://api.firecrawl.dev/v1/scrape"
    query = f"{requirements.get('material', '')} suppliers in {requirements.get('region', '')}"
    search_url = f"https://www.google.com/search?q={query.replace(' ', '+')}"

    headers = {
        "Authorization": f"Bearer {FIRECRAWL_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {"url": search_url, "options": {"waitUntil": "domcontentloaded"}}

    try:
        res = requests.post(FIRECRAWL_URL, json=payload, headers=headers)
        res.raise_for_status()
        data = res.json()
        return [{"source": "firecrawl", "scraped_content": data.get("content", "No content found"), "reputation_score": 3}]
    except Exception as e:
        return [{"source": "firecrawl", "error": str(e)}]

# 7. Supply Exploration Agent
def supply_exploration_agent(state: CreateChainState):
    mongo_results = query_mongodb(state["requirements"], state.get("esg_preference"))
    firecrawl_results = firecrawl_scrape_tool(state["requirements"])
    suppliers = mongo_results + firecrawl_results
    return {"suppliers": suppliers, "exploration_results": suppliers}

# 8. SupplyChainSageAgent
def supply_chain_sage_agent(state: CreateChainState):
    suppliers = state.get("suppliers", [])
    requirements = state.get("requirements", {})
    region = requirements.get("region", "").lower()
    max_days = requirements.get("delivery_time_days", {}).get("$lte", 30)
    esg_pref = state.get("esg_preference", "medium").lower()

    evaluated = []
    for s in suppliers:
        score = 0
        reasons, warnings = [], []

        esg = s.get("esg_score", "low").lower()
        if esg_pref == "high" and esg in ["high", "medium"]:
            score += 1
            reasons.append("ESG score meets preference")
        elif esg_pref == "medium" and esg in ["medium", "high"]:
            score += 1
            reasons.append("ESG score acceptable")
        elif esg_pref == "low":
            score += 1
        else:
            warnings.append(f"ESG score '{esg}' not ideal")

        if s.get("delivery_time_days", 999) <= max_days:
            score += 1
            reasons.append("Delivery time acceptable")
        else:
            warnings.append("Late delivery")

        if isinstance(s.get("price"), (int, float)):
            score += 1
            reasons.append("Price available")
        else:
            warnings.append("Missing price")

        if region in s.get("region", "").lower():
            score += 1
            reasons.append("Region match")
        else:
            warnings.append("Region mismatch")

        if s.get("certifications"):
            score += 1
            reasons.append("Certifications found")
        else:
            warnings.append("No certifications")

        if s.get("reputation_score", 0) >= 3 or s.get("source") == "firecrawl":
            score += 1
            reasons.append("Credible supplier")
        else:
            warnings.append("Low reputation")

        evaluated.append({"supplier": s, "score": score, "reasons": reasons, "warnings": warnings})

    top = sorted(evaluated, key=lambda x: x["score"], reverse=True)[:5]
    valid = [s for s in top if s["score"] >= 3]

    if len(valid) >= 2:
        return {"evaluation_feedback": "good", "suppliers": valid}
    else:
        return {"evaluation_feedback": "not good enough"}

# 9. Final Report Agent
def generate_final_report(state: CreateChainState):
    session_id = state["session_id"]
    if state["evaluation_feedback"] == "good":
        report = {
            "status": "success",
            "summary": f"{len(state['suppliers'])} top suppliers selected.",
            "top_suppliers": state["suppliers"],
            "evaluation_criteria": ["esg_score", "delivery_time_days", "price", "region", "certifications", "reputation_score"],
            "min_score_required": 3,
            "max_score_possible": 6
        }
        report_collection.insert_one({"session_id": session_id, "report": report, "timestamp": datetime.utcnow()})
        log_message("assistant", str(report), session_id)
        return {"final_report": report}
    else:
        retry_msg = "Supply chain not good enough. Reanalyzing."
        log_message("assistant", retry_msg, session_id)
        return {"final_report": {"status": "retry", "message": retry_msg}}

# 10. Build LangGraph
builder = StateGraph(CreateChainState)
builder.add_node("parse", parse_requirements)
builder.add_node("explore", supply_exploration_agent)
builder.add_node("evaluate", supply_chain_sage_agent)
builder.add_node("report", generate_final_report)

builder.add_edge(START, "parse")
builder.add_edge("parse", "explore")
builder.add_edge("explore", "evaluate")

builder.add_conditional_edges("evaluate", lambda s: "retry" if s["evaluation_feedback"] == "not good enough" else "final", {
    "retry": "parse",
    "final": "report"
})
builder.add_edge("report", END)

create_chain_agent = builder.compile()

# 11. Sample Run
if __name__ == "__main__":
    user_input_dict = {
        "material": "copper",
        "region": "Asia",
        "delivery_time_days": {"$lte": 15}
    }

    initial_state = CreateChainState(
        session_id="session_001",
        raw_input=user_input_dict,
        esg_preference="high",
        requirements={}, suppliers=[], exploration_results=[], evaluation_feedback="", final_report={}
    )

    result = create_chain_agent.invoke(initial_state)
    print("\nFinal Output:")
    print(result["final_report"])

# 12. Sample MongoDB supplier seed data (run separately to populate the DB)
def seed_sample_suppliers():
    sample_suppliers = [
        {
            "name": "Asia Metals Ltd",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 10,
            "esg_score": "high",
            "price": 150,
            "certifications": ["ISO 9001"],
            "reputation_score": 4
        },
        {
            "name": "Green Copper Co",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 12,
            "esg_score": "medium",
            "price": 145,
            "certifications": ["ISO 14001"],
            "reputation_score": 3
        },
        {
            "name": "EcoMinerals",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 20,
            "esg_score": "low",
            "price": 130,
            "reputation_score": 2
        },
        {
            "name": "Trusted Copper Supplies",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 9,
            "esg_score": "high",
            "price": 160,
            "certifications": ["RoHS"],
            "reputation_score": 5
        },
        {
            "name": "Budget Copper Traders",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 15,
            "esg_score": "medium",
            "price": 120,
            "reputation_score": 3
        }
    ]
    suppliers_collection.insert_many(sample_suppliers)

# To seed the database, uncomment and run:
# seed_sample_suppliers()
